In [9]:
import streamlit as st
import pandas as pd
import pdfplumber
import numpy as np
import re


def extract_table_from_pdf(pdf_path, has_header):
    dataframe_list = []
    header = None

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            tables = page.extract_tables(
                {"vertical_strategy": "lines", "horizontal_strategy": "lines"}
            )

            for table_index, table in enumerate(tables):
                if has_header:
                    if len(table) > 1:  # Ensure there is at least one row of data
                        header = table[0] + ["page"]
                        for row in table[1:]:
                            if len(row) < len(header) - 1:
                                row.extend([None] * (len(header) - 1 - len(row)))
                            elif len(row) > len(header) - 1:
                                row = row[: len(header) - 1]
                            row.append(page_num)
                        df = pd.DataFrame(table[1:], columns=header)
                        dataframe_list.append(df)
                else:
                    header = [f"column_{i}" for i in range(len(table[0]))] + ["page"]
                    for row in table:
                        if len(row) < len(header) - 1:
                            row.extend([None] * (len(header) - 1 - len(row)))
                        elif len(row) > len(header) - 1:
                            row = row[: len(header) - 1]
                        row.append(page_num)
                    df = pd.DataFrame(table, columns=header)
                    dataframe_list.append(df)

    if dataframe_list:
        try:
            df_extracted = pd.concat(dataframe_list)
            df_extracted = df_extracted.replace("", None)
            return df_extracted
        except AssertionError as e:
            return pd.DataFrame()
    else:
        return (
            pd.DataFrame()
        )  # Return an empty DataFrame if no tables were extracted or matched


def update_headers(column_mappings, df):
    # Rename the columns using the provided column mapping
    df.rename(columns=column_mappings, inplace=True)

    return df


def clean_dataframe(df):
    if "vrm.1" in df.columns:
        df["vrm"] = np.where(df["vrm"].isnull(), df["vrm.1"], df["vrm"])
        df["vrm"] = df["vrm"].fillna(method="ffill")
        df.drop(columns=["vrm.1"], inplace=True)
        df.drop_duplicates(inplace=True, subset=["vrm"])
        df.reset_index(inplace=True, drop=True)

    for column in ["make", "model"]:
        if f"{column}.1" in df.columns:
            df[column] = df[column].combine_first(df[f"{column}.1"])
            df.drop(columns=[f"{column}.1"], inplace=True)

    if "vrm" in df.columns:
        df = df.copy()[
            df["vrm"].apply(lambda x: re.match(r"^[A-Z0-9\s]*$", str(x)) is not None)
        ]

    return df


council = "Renfrewshire"
council_data = {
    "Argyll and Bute": {
        "id": 1,
        "has_header": False,
        "has_gridlines": True,
        "column_mappings": {
            "column_0": "vrm",
            "column_1": "make",
            "column_2": "model",
        },
    },
    "BCP": {
        "id": 2,
        "has_header": True,
        "has_gridlines": False,
        "column_mappings": {
            "Vehicle Reg": "vrm",
            "Make / model": "make",
        },
    },
    "Salford": {
        "id": 3,
        "has_header": True,
        "has_gridlines": True,
        "column_mappings": {"REG": "vrm", "VEHICLE TYPE": "make"},
    },
    "Renfrewshire": {
        "id": 4,
        "has_header": False,
        "has_gridlines": True,
        "column_mappings": {"column_1": "vrm", "column_2": "make"},
    },
    "East Lothian": {
        "id": 5,
        "has_header": False,
        "has_gridlines": True,
        "column_mappings": {
            "column_0": "vrm",
            "column_1": "vrm.1",
            "column_3": "make",
            "column_4": "make.1",
            "column_6": "model",
            "column_7": "model.1",
        },
    },
}
has_header = council_data[council]["has_header"]

pdf_path = f"pdfs/gridlines/{council}.pdf"
df = extract_table_from_pdf(pdf_path, has_header=has_header)
df = update_headers(council_data[council]["column_mappings"], df)
df_clean = clean_dataframe(df)


In [10]:
df_clean

,column_0,vrm,make,page
1,PH0500,R13 NDR,Kia Niro,1
2,PH0501,HY18 DWV,Mercedes Benz E220d,1
3,PH0503,BD69 OGX,Skoda Octavia,1
4,PH0504,FN67 KGO,Kia Optima,1
5,PH0505,ST21 GZB,Skoda Octavia,1
...,...,...,...,...
0,SURR TX075,SF13 NLA,Peugot Expert E7,19
1,SURR PH1530,SK22 VGA,MG MG5 Excite EV,19
2,SURR PH0838,SA66 ZXN,Toyota Avensis,19
3,SURR PH0732,SF17 XVH,Skoda Octavia,19
